In [ ]:
import os.path as op
import pickle

import nibabel as nib
import numpy as np
from nibabel import GiftiImage
from nibabel.gifti import GiftiDataArray
from surfplot.utils import add_fslr_medial_wall
from surfplot.utils import add_fslr_medial_wall
from surfplot import Plot
from neuromaps.datasets import fetch_fslr
from sklearn.metrics import pairwise_distances

In [ ]:
gradients = np.load("../results/gradient/gradients.npy")

template_dir = "../data/templates"
subcortical_fn = op.join(template_dir, "rois-subcortical_mni152_mask.nii.gz")
subcort_img = nib.load(subcortical_fn)

full_vertices = 64984
hemi_vertices = full_vertices // 2

subcort_dat = subcort_img.get_fdata()
subcort_mask = subcort_dat != 0
n_subcort_vox = np.where(subcort_mask)[0].shape[0]

n_gradients = gradients.shape[1]
grad_lst = []
for i in range(n_gradients):
    cort_grads = gradients[: gradients.shape[0] - n_subcort_vox, i]
    grad_lst.append(cort_grads)

grad_arr = np.array(grad_lst).T

In [ ]:
surfaces = fetch_fslr()
lh, rh = surfaces['inflated']
sulc_lh, sulc_rh = surfaces['sulc']

full_vertices = 64984
hemi_vertices = full_vertices // 2

In [ ]:
max_peak, min_peak = grad_arr[:,0].max(), grad_arr[:,0].min()

sigma_0 = 1
final_dict = {}
for method in ["PCT", "KMeans", "KDE"]:
    with open(f"../results/segmentation/{method}_results.pkl", "rb") as results_file:
        seg_dict = pickle.load(results_file)
    
    segments = seg_dict["segments"]
    peaks = seg_dict["peaks"]
    grad_segments = []   
    for segment, peak in zip(segments, peaks):

        peak[0], peak[-1] = min_peak, max_peak
        grad_maps = []
        for grad_map, peak_act in zip(segment, peak):
            peak_coord = np.array(peak_act)
            indices = np.where(grad_map!=0)[0]
            grad_val = grad_map[indices]
            distances = pairwise_distances(
                grad_val.reshape(-1, 1), 
                peak_coord.reshape(1, -1), 
                metric="euclidean"
            ).flatten()

            mean_dist = np.mean(distances)
            sigma = mean_dist * sigma_0
            affinity =  np.exp(-distances**2 / (2 * sigma**2))

            pseudo_act_map = np.zeros_like(grad_map, dtype=float)
            pseudo_act_map[indices] = np.array(affinity)


            grad_maps.append(pseudo_act_map)
        grad_segments.append(grad_maps)
    final_dict[method] = grad_segments

with open("/Users/jperaza/Documents/GitHub/gradient-decoding/results/segmentation/segments.pkl", "wb") as segmentation_file:
    pickle.dump(final_dict, segmentation_file)


In [ ]:
final_dict

In [ ]:
with open("/Users/jperaza/Documents/GitHub/gradient-decoding/results/segmentation/segments.pkl", "wb") as segmentation_file:
    pickle.dump(final_dict, segmentation_file)